In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from uuid import UUID

import matplotlib.pyplot as plt

import sys
sys.path.append('/Users/mallen2/alternate_branches/eval-compatible-server/e-mission-server')

import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.trip_queries as esdtq
import emission.core.wrapper.user as ecwu

import statistics

import confusion_matrix_handling as cm_handling
from confusion_matrix_handling import MODE_MAPPING_DICT
import get_EC

import sklearn.model_selection as skm

METERS_TO_MILES = 0.000621371 # 1 meter = 0.000621371 miles

df_EI = pd.read_csv(r'Public_Dashboard/auxiliary_files/energy_intensity.csv')

In [ ]:
# maybe later I could get all trips at once? # get_non_user_time_series() ?

all_users = esta.TimeSeries.get_uuid_list()
confirmed_trip_df_map = {}
labeled_trip_df_map = {}
expanded_labeled_trip_df_map = {}
expanded_all_trip_df_map = {}
for u in all_users:
    ts = esta.TimeSeries.get_time_series(u)
    ct_df = ts.get_data_df("analysis/confirmed_trip")

    confirmed_trip_df_map[u] = ct_df
    labeled_trip_df_map[u] = esdtq.filter_labeled_trips(ct_df)
    expanded_labeled_trip_df_map[u] = esdtq.expand_userinputs(
        labeled_trip_df_map[u])
    expanded_all_trip_df_map[u] = esdtq.expand_userinputs(
        confirmed_trip_df_map[u])

In [ ]:
all_labeled_trips = pd.concat(labeled_trip_df_map.values(), ignore_index=True)
#all_labeled_no_air = 

In [ ]:
# Base mode map for GIS
gis_sensed_modes = {0 : 'no_sensed',    # UNKNOWN  #NOTE: this is important info to mention.
        1 : 'walking',    # WALKING
        2 : 'bicycling',    # BICYCLING
        3 : 'bus',        # BUS
        4 : 'train',      # TRAIN
        5 : 'car',        # CAR
        6 : 'air_or_hsr', # AIR_OR_HSR
        7 : 'subway',      # SUBWAY
        8 : 'train',      # TRAM
        9 : 'train',      # LIGHT_RAIL
}

# Error related info
unit_dist_MCS_df = pd.read_csv("unit_distance_MCS.csv").set_index("moment")
android_EI_moments_df = pd.read_csv("android_EI_moments.csv").set_index("mode")
ios_EI_moments_df = pd.read_csv("ios_EI_moments.csv").set_index("mode")

# Dictionary of energy intensities in kWH/PMT
energy_dict = cm_handling.get_energy_dict(df_EI)


# sensed_car -> (via MODE_MAPPING_DICT) “Gas Car, sensed” in energy dict, 
# which is used for the ground truth car intensity in get_conditional_EI_expectation_and_variance(). 
# Then the sensed mode will show car, but the EI used will be based on a car with a 1.5 person load factor.
drove_alone_EI = energy_dict["Gas Car, drove alone"]
load_factor = 1#1.5
energy_dict.update({"Gas Car, sensed": drove_alone_EI/load_factor})

expanded_labeled_trips = pd.concat(expanded_labeled_trip_df_map.values(), ignore_index=True)

In [ ]:
def investigate_sensing_for_one_user_label(expanded_labeled_trips_df, label):
    one_label = expanded_labeled_trips[expanded_labeled_trips['mode_confirm'] == label]

    print(get_aggregate_EC(one_label,True, unit_dist_MCS_df, android_EI_moments_df, ios_EI_moments_df, gis_sensed_modes,energy_dict))
    label_sensed = []
    label_dist = []
    for _,ct in one_label.iterrows():
        label_sensed += [m for m in ct['section_modes']]
        label_dist += [d for d in ct['section_distances']]
    return pd.DataFrame({"mode": label_sensed, "dist": label_dist})

In [ ]:
investigate_sensing_for_one_user_label(expanded_labeled_trips,'shared_ride')

In [ ]:
pd.DataFrame({"mode": snowboard_sensed, "dist": snowboard_dist})

In [ ]:
energy_dict

In [ ]:
android_EI_moments_df

In [ ]:
# Find the distance in each mode.
mode_distance_df = expanded_labeled_trips[['mode_confirm','distance']].groupby("mode_confirm").sum()
mode_distance_df.loc['car'] + mode_distance_df.loc['drove_alone'] - mode_distance_df.loc["shared_ride"]

# More distance in shared ride than in drove alone!

In [ ]:
mode_dist = expanded_labeled_trips[['mode_confirm','distance']].groupby("mode_confirm").sum()

In [ ]:
mode_dist.loc['taxi']* METERS_TO_MILES * (energy_dict['Taxi/Uber/Lyft'] - android_EI_moments_df["mean(EI)"]["car"])

In [ ]:
mode_dist

In [ ]:
expanded_labeled_trips["mode_confirm"].value_counts()

In [ ]:
sensed_mode_dict = {}
car_user_sensing_match = 0

car_walking_cases = []
car_trip_n_sections = []
car_biking_cases = []

walks = []
cars = 0

multi_section_cars = []

# get all cases where car is not the only section.

for _,ct in expanded_labeled_trips.iterrows():
    longest_section = max(ct["section_distances"])
    primary_mode = ct["section_modes"][ct["section_distances"]==longest_section]

    # in case there are ever tied longest sections.
    # pick the most energy intensive mode.
    if isinstance(primary_mode,list): 
        mini_energy_dict = {x:energy_dict[MODE_MAPPING_DICT[x]] for x in primary_mode}
        primary_mode = max(mini_energy_dict, key=mini_energy_dict.get)

    if primary_mode == 'car': cars += 1

    if primary_mode == "car" and ct["mode_confirm"] in ["drove_alone","shared_ride","car"]:

        modes = ct["section_modes"]
        dists = ct["section_distances"]

        if len(modes) > 1:
            multi_section_cars.append(ct)

        if ct["section_modes"] == ['car', 'walking']:
            car_walking_cases.append(ct["section_distances"])
        if ct["section_modes"] == ['car', 'bicycling']:
            car_biking_cases.append(ct["section_distances"])
        elif ['car','walking'] in ct["section_modes"]:
            print(ct["section_modes"])

        #print(ct["section_modes"])
        #print(ct["section_distances"])
        car_user_sensing_match +=1

        car_trip_n_sections.append(len(ct["section_distances"]))


    if primary_mode not in sensed_mode_dict: sensed_mode_dict[primary_mode] = 1
    sensed_mode_dict[primary_mode] +=1

In [ ]:
# Add primary mode and length columns to expanded labeled trips
primary_modes = []
primary_lengths = []

for _,ct in expanded_labeled_trips.iterrows():
    # Get primary mode
    longest_section = max(ct["section_distances"])
    primary_mode = ct["section_modes"][ct["section_distances"]==longest_section]

    # in case there are ever tied longest sections.
    # pick the most energy intensive mode.
    if isinstance(primary_mode,list): 
        mini_energy_dict = {x:energy_dict[MODE_MAPPING_DICT[x]] for x in primary_mode}
        primary_mode = max(mini_energy_dict, key=mini_energy_dict.get)

    primary_modes.append(primary_mode)
    primary_lengths.append(longest_section)

expanded_labeled_trips['primary_mode'] = primary_modes
expanded_labeled_trips['primary_length'] = primary_lengths

In [ ]:
user_label_to_sensing_map = {
    'drove_alone': 'car',
    'shared_ride': 'car',
    'pilot_ebike': 'bicycling',
    'walk': 'walking',
    'bus': 'bus',
    'not_a_trip': 'no_sensed',
    'car': 'car',
    'taxi': 'car',
    'bike': 'bicycling',
    'train': 'train',
    'subway': 'subway',
    'air': 'air_or_hsr'
}

gis_sensed_modes = {0 : 'no_sensed',    # UNKNOWN  #NOTE: this is important info to mention.
        1 : 'walking',    # WALKING
        2 : 'bicycling',    # BICYCLING
        3 : 'bus',        # BUS
        4 : 'train',      # TRAIN
        5 : 'car',        # CAR
        6 : 'air_or_hsr', # AIR_OR_HSR
        7 : 'subway',      # SUBWAY
        8 : 'train',      # TRAM
        9 : 'train',      # LIGHT_RAIL
}

In [ ]:
import sklearn.metrics as sklearn_metrics

confusion_ready = expanded_labeled_trips.dropna()
user_labels = confusion_ready['mode_confirm']
sensed_labels = confusion_ready['primary_mode']
cm_user_v_sensed = sklearn_metrics.confusion_matrix(user_labels,sensed_labels)

In [ ]:
cm_user_v_sensed.shape

In [ ]:
mc_types = [type(x) for x in expanded_labeled_trips['mode_confirm']]
type(confusion_ready['mode_confirm'].iloc[mc_types.index(float)])

# so mode confirm has an nan.

In [ ]:
sensed_user = expanded_labeled_trips[['primary_mode','mode_confirm']]
sensed_user[sensed_user['mode_confirm']== 'electric_vehicle']  # sensed as walking!

In [ ]:
expanded_labeled_trips['mode_confirm'].value_counts()

In [ ]:
length_diffs = []
# Verify that section sums are equal to trip lengths.
for _,ct in expanded_labeled_trips.iterrows():
    section_sum = sum(ct["section_distances"])
    trip_length = ct["distance"]

    length_diffs.append(trip_length-section_sum)
    #np.testing.assert_approx_equal(section_sum,ct['distance'],5)
plt.hist(length_diffs)

In [ ]:
primary_mode = ["walking","car"]
mini_energy_dict = {x:energy_dict[MODE_MAPPING_DICT[x]] for x in primary_mode}
primary_mode = max(mini_energy_dict, key=mini_energy_dict.get)

In [ ]:
car_user_sensing_match/cars  #P(userlabel = car| predict car) = P(predict and ground truth car)/P(predict car)

In [ ]:
walk_sum = 0
for trip in car_walking_cases:
    print(trip)
    walk_sum += trip[1]

In [ ]:
# Increase in EC if we count walk sections for primary car trips as car.
walk_sum*1.13*1.04*METERS_TO_MILES - walk_sum*0.01*1.04*METERS_TO_MILES

In [ ]:
expanded_labeled_trips.shape
    longest_section = max(ct["section_distances"])
    primary_mode = ct["section_modes"][ct["section_distances"]==longest_section]

    # in case there are ever tied longest sections.
    # pick the most energy intensive mode.
    if isinstance(primary_mode,list): 
        mini_energy_dict = {x:energy_dict[MODE_MAPPING_DICT[x]] for x in primary_mode}
        primary_mode = max(mini_energy_dict, key=mini_energy_dict.get)

In [ ]:
def get_EI_moments_for_trip(mode,os,android_EI_moments,ios_EI_moments):
    # also works for a section.
    if os == "android":
        mean_EI = android_EI_moments["mean(EI)"][mode]
        var_EI = android_EI_moments["variance(EI)"][mode]  # variance given the inferred mode is <mode>
    elif os == "ios":
        mean_EI = ios_EI_moments["mean(EI)"][mode]
        var_EI = ios_EI_moments["variance(EI)"][mode]

    return mean_EI, var_EI

def get_aggregate_EC(trips_df, only_sensing, unit_dist_MCS, android_EI_moments, ios_EI_moments,gis_sensed_modes, energy_dict):

    mean_EC_agg = 0    # aggregate energy consumption
    var_EC_agg = 0

    for  _,ct in trips_df.iterrows():
        # Get operating system
        u = ecwu.User(ct.user_id)
        os = u.getProfile()["curr_platform"]

        # Get OS specific trip length info.
        mean_for_unit_L = unit_dist_MCS_df[os]["mean"]
        var_for_unit_L = unit_dist_MCS_df[os]["var"]


        # Get trip mode info.
        # Later the condition will be whether the model chosen is sensing.
        if only_sensing == True:
            # Get segments for the trip.
            n_sections = len(ct["section_modes"])
            section_modes = ct["section_modes"]
            sections_lengths = np.array(ct["section_distances"])*METERS_TO_MILES   # 1 meter = 0.000621371 miles

            mean_L = sections_lengths*mean_for_unit_L
            var_L = sections_lengths**2 * var_for_unit_L
            
            
            for s in range(0,n_sections):
                # EI mean and variance.
                # Perhaps it would be better to keep the moments in the same file?

                if section_modes[s] == "air_or_hsr": continue

                # Later: switch to a map style function.
                mean_EI, var_EI = get_EI_moments_for_trip(section_modes[s],os,android_EI_moments,ios_EI_moments)

                # Propagate variance for the trip
                mean_EC = mean_L[s]*mean_EI
                var_EC = var_EI*mean_L[s]**2 + var_L[s]*mean_EI**2

                # Add to total - follows from assumed independence of section errors.
                mean_EC_agg += mean_EC
                var_EC_agg += var_EC
        
        # use user labels.
        else:
            mode = ct["mode_confirm"]  # need to make sure you convert it to an appropriate energy intensity.

            if mode not in MODE_MAPPING_DICT or mode == np.nan: continue
            if MODE_MAPPING_DICT[mode] == "Air": continue
            EI = energy_dict[MODE_MAPPING_DICT[mode]]

            length = ct["distance"]*METERS_TO_MILES
            mean_L = length* mean_for_unit_L  
            var_L = length**2 * var_for_unit_L

            mean_EC_agg += EI*mean_L
            var_EC_agg +=  EI*var_L

    #print(air_count)
    return mean_EC_agg, var_EC_agg

In [ ]:
def get_primary_mode_aggregate_EC(trips_df, only_sensing, unit_dist_MCS, android_EI_moments, ios_EI_moments,gis_sensed_modes, energy_dict, using_predictions, only_primary_section):

    mean_EC_agg = 0    # aggregate energy consumption
    var_EC_agg = 0
    primary_mode_dict = {}

    for  _,ct in trips_df.iterrows():
        # Get operating system
        u = ecwu.User(ct.user_id)
        os = u.getProfile()["curr_platform"]

        # Get OS specific trip length info.
        mean_for_unit_L = unit_dist_MCS_df[os]["mean"]
        var_for_unit_L = unit_dist_MCS_df[os]["var"]

        # Get primary mode
        longest_section = max(ct["section_distances"])
        primary_mode = ct["section_modes"][ct["section_distances"]==longest_section]

        # in case there are ever tied longest sections.
        # pick the most energy intensive mode.
        if isinstance(primary_mode,list): 
            mini_energy_dict = {x:energy_dict[MODE_MAPPING_DICT[x]] for x in primary_mode}
            primary_mode = max(mini_energy_dict, key=mini_energy_dict.get)

        if primary_mode == 'air_or_hsr': continue

        # Get the trip length or primary section length.
        if only_primary_section == True:
            trip_length = longest_section*METERS_TO_MILES
        else:
            trip_length = ct['distance']*METERS_TO_MILES   # 1 meter = 0.000621371 miles

        # Get trip mode info.
        # Later the condition will be whether the model chosen is sensing.
        if only_sensing == True:

            # save trip length sums for each mode
            if primary_mode not in primary_mode_dict: primary_mode_dict[primary_mode] = 0
            primary_mode_dict[primary_mode] +=  trip_length

            mean_L = trip_length*mean_for_unit_L
            var_L = trip_length**2 * var_for_unit_L




            if using_predictions:
                mean_EI = energy_dict[MODE_MAPPING_DICT[primary_mode]]
                var_EI = 0
            else:
                # Later: switch to a map style function.
                mean_EI, var_EI = get_EI_moments_for_trip(primary_mode,os,android_EI_moments,ios_EI_moments)

            # Propagate variance for the trip
            mean_EC = mean_L*mean_EI
            var_EC = var_EI*mean_L**2 + var_L*mean_EI**2

            # Add to total - follows from assumed independence of section errors.
            mean_EC_agg += mean_EC
            var_EC_agg += var_EC
        
        # use user labels.
        else:
            mode = ct["mode_confirm"]  # need to make sure you convert it to an appropriate energy intensity.

            if mode not in MODE_MAPPING_DICT or mode == np.nan: continue
            if MODE_MAPPING_DICT[mode] == "Air": continue
            EI = energy_dict[MODE_MAPPING_DICT[mode]]

            length = trip_length
            mean_L = length* mean_for_unit_L  
            var_L = length**2 * var_for_unit_L

            mean_EC_agg += EI*mean_L
            var_EC_agg +=  EI*var_L

    #print(air_count)
    return mean_EC_agg, var_EC_agg, primary_mode_dict

In [ ]:
# In this cell: 
# The primary section is the only section used.
# I ignore shared rides.
# Car load factor to create the EI moments dataframe: 1
# Car precision is increased stepwise by 0.1, with bus taking the rest of the labels. (see store_errors.ipynb).
# In this analysis, the adjusted car precision is the same for both ios and android.

# drop shared rides.
elt = expanded_labeled_trips.copy()
no_shared_rides = elt.drop(elt[elt.mode_confirm == "shared_ride"].index)

prediction_EC_primary_section = get_primary_mode_aggregate_EC(no_shared_rides,True, unit_dist_MCS_df, 
        android_EI_moments_no_shared_ride_df, # these do not get used when we only use the prediction.
        ios_EI_moments_no_shared_ride_df, 
        gis_sensed_modes,energy_dict,
        using_predictions=True,
        only_primary_section=True)

for j in range(0,4):
    print(f"car precision: {0.45+ 0.1*(j+1)}")
    precision_adj_android_EI_moments_df = pd.read_csv("android_EI_moments_no_shared_ride_car_precision_adjustment_"+ str(j) + ".csv").set_index("mode")
    precision_adj_ios_EI_moments_df = pd.read_csv("ios_EI_moments_no_shared_ride_car_precision_adjustment_"+ str(j) + ".csv").set_index("mode")

    # Looking only at the primary section EC consumption and applying the user or sensed label to it.
    confusion_EC_primary_section = get_primary_mode_aggregate_EC(no_shared_rides,True, unit_dist_MCS_df, 
            precision_adj_android_EI_moments_df, 
            precision_adj_ios_EI_moments_df, 
            gis_sensed_modes,energy_dict,
            using_predictions=False,
            only_primary_section=True)

    print(f"prediction,confusion based: {prediction_EC_primary_section[0],confusion_EC_primary_section[0]}")

user_no_shared_EC_primary_section = get_primary_mode_aggregate_EC(no_shared_rides,False, unit_dist_MCS_df, 
        android_EI_moments_no_shared_ride_df, # these do not get used if looking at user labels.
        ios_EI_moments_no_shared_ride_df, 
        gis_sensed_modes,energy_dict,
        using_predictions=False,
        only_primary_section=True)
print(f"User EC: {user_no_shared_EC_primary_section[0]}")

In [ ]:
# In this cell: 
# The primary mode is applied to the entire trip.
# I ignore shared rides.
# Car load factor to create the EI moments dataframe: 1
# Car precision is increased stepwise by 0.1, with bus taking the rest of the labels. (see store_errors.ipynb).
# In this analysis, the adjusted car precision is the same for both ios and android.

# Ensure that a predicted car trip is treated as drove alone
drove_alone_EI = energy_dict["Gas Car, drove alone"]
load_factor = 1
energy_dict.update({"Gas Car, sensed": drove_alone_EI/load_factor})

# drop shared rides.
elt = expanded_labeled_trips.copy()
no_shared_rides = elt.drop(elt[elt.mode_confirm == "shared_ride"].index)

prediction_EC_primary_section = get_primary_mode_aggregate_EC(no_shared_rides,True, unit_dist_MCS_df, 
        android_EI_moments_no_shared_ride_df, # these do not get used when we only use the prediction.
        ios_EI_moments_no_shared_ride_df, 
        gis_sensed_modes,energy_dict,
        using_predictions=True,
        only_primary_section=False)

for j in range(0,4):
    print(f"car precision: {0.45+ 0.1*(j+1)}")
    precision_adj_android_EI_moments_df = pd.read_csv("android_EI_moments_no_shared_ride_car_precision_adjustment_"+ str(j) + ".csv").set_index("mode")
    precision_adj_ios_EI_moments_df = pd.read_csv("ios_EI_moments_no_shared_ride_car_precision_adjustment_"+ str(j) + ".csv").set_index("mode")

    # Looking only at the primary section EC consumption and applying the user or sensed label to it.
    confusion_EC_primary_section = get_primary_mode_aggregate_EC(no_shared_rides,True, unit_dist_MCS_df, 
            precision_adj_android_EI_moments_df, 
            precision_adj_ios_EI_moments_df, 
            gis_sensed_modes,energy_dict,
            using_predictions=False,
            only_primary_section=False)

    print(f"prediction,confusion based: {prediction_EC_primary_section[0],confusion_EC_primary_section[0]}")

user_no_shared_EC_primary_section = get_primary_mode_aggregate_EC(no_shared_rides,False, unit_dist_MCS_df, 
        android_EI_moments_no_shared_ride_df, # these do not get used if looking at user labels.
        ios_EI_moments_no_shared_ride_df, 
        gis_sensed_modes,energy_dict,
        using_predictions=False,
        only_primary_section=False)
print(f"User EC: {user_no_shared_EC_primary_section[0]}")

In [ ]:
# In this cell: 
# All sections are used.
# I ignore shared rides.
# Car load factor to create the EI moments dataframe: 1
# Car precision is increased stepwise by 0.1, with bus taking the rest of the labels. (see store_errors.ipynb).
# In this analysis, the adjusted car precision is the same for both ios and android.

# Ensure that a predicted car trip is treated as drove alone
drove_alone_EI = energy_dict["Gas Car, drove alone"]
load_factor = 1
energy_dict.update({"Gas Car, sensed": drove_alone_EI/load_factor})

# drop shared rides.
elt = expanded_labeled_trips.copy()
no_shared_rides = elt.drop(elt[elt.mode_confirm == "shared_ride"].index)

mean_EC_naive_no_shared, var_EC_naive_no_shared, avg_EI_no_shared= get_aggregate_EC_with_extras(no_shared_rides,True, unit_dist_MCS_df, 
        android_EI_moments_df, # not used in prediction based calculation.
        ios_EI_moments_df, 
        gis_sensed_modes,energy_dict,
        use_naive_sensing_prediction=True,
        car_load_factor = 1)

for j in range(0,4):
    print(f"car precision: {0.45+ 0.1*(j+1)}")
    precision_adj_android_EI_moments_df = pd.read_csv("android_EI_moments_no_shared_ride_car_precision_adjustment_"+ str(j) + ".csv").set_index("mode")
    precision_adj_ios_EI_moments_df = pd.read_csv("ios_EI_moments_no_shared_ride_car_precision_adjustment_"+ str(j) + ".csv").set_index("mode")


    mean_EC_no_shared_expected, var_EC_no_shared_expected = get_aggregate_EC(no_shared_rides,True, unit_dist_MCS_df,
            precision_adj_android_EI_moments_df, 
            precision_adj_ios_EI_moments_df, 
            gis_sensed_modes,energy_dict)

    print(f"prediction,confusion based: {mean_EC_naive_no_shared,mean_EC_no_shared_expected}")


mean_EC_no_shared_users,_ = get_aggregate_EC(no_shared_rides,False, unit_dist_MCS_df,
        precision_adj_android_EI_moments_df, 
        precision_adj_ios_EI_moments_df, 
        gis_sensed_modes,energy_dict)
print(f"User EC: {mean_EC_no_shared_users}")

In [ ]:
# A look at the values for primary sections when we have no shared rides and the confusion matrices in Gabe's paper.
# ie load factor 1 for sensed car.
android_EI_moments_no_shared_ride_df = pd.read_csv("android_EI_moments_no_shared_ride.csv").set_index("mode")
ios_EI_moments_no_shared_ride_df = pd.read_csv("ios_EI_moments_no_shared_ride.csv").set_index("mode")

# drop shared rides.
elt = expanded_labeled_trips.copy()
no_shared_rides = elt.drop(elt[elt.mode_confirm == "shared_ride"].index)

prediction_no_shared_EC_primary_section = get_primary_mode_aggregate_EC(no_shared_rides,True, unit_dist_MCS_df, 
        android_EI_moments_no_shared_ride_df, 
        ios_EI_moments_no_shared_ride_df, 
        gis_sensed_modes,energy_dict,
        using_predictions=True,
        only_primary_section=True)

confusion_no_shared_EC_primary_section = get_primary_mode_aggregate_EC(no_shared_rides,True, unit_dist_MCS_df, 
        android_EI_moments_no_shared_ride_df, 
        ios_EI_moments_no_shared_ride_df, 
        gis_sensed_modes,energy_dict,
        using_predictions=False,
        only_primary_section=True)

user_no_shared_EC_primary_section = get_primary_mode_aggregate_EC(no_shared_rides,False, unit_dist_MCS_df, 
        android_EI_moments_no_shared_ride_df, 
        ios_EI_moments_no_shared_ride_df, 
        gis_sensed_modes,energy_dict,
        using_predictions=False,
        only_primary_section=True)

print("prediction, confusion EC for no shared rides and only primary sections")
prediction_no_shared_EC_primary_section[0],confusion_no_shared_EC_primary_section[0], user_no_shared_EC_primary_section[0]

In [ ]:
android_EI_moments_no_shared_ride_df = pd.read_csv("android_EI_moments_no_shared_ride.csv").set_index("mode")
ios_EI_moments_no_shared_ride_df = pd.read_csv("ios_EI_moments_no_shared_ride.csv").set_index("mode")
android_EI_moments_no_shared_ride_df,ios_EI_moments_no_shared_ride_df

In [ ]:
prediction_EC_primary_section[0],user_EC[0],confusion_EC_primary_section[0]

In [ ]:
np.diff([prediction_EC_primary_section[0],user_EC[0],confusion_EC_primary_section[0]])

In [ ]:
#prediction_EC_primary_trip = get_primary_mode_aggregate_EC(expanded_labeled_trips,True, unit_dist_MCS_df, android_EI_moments_df, ios_EI_moments_df, gis_sensed_modes,energy_dict,using_predictions=True,only_primary_section=False)
#confusion_EC_primary_trip = get_primary_mode_aggregate_EC(expanded_labeled_trips,True, unit_dist_MCS_df, android_EI_moments_df, ios_EI_moments_df, gis_sensed_modes,energy_dict,False,False)
#user_EC_trip = get_primary_mode_aggregate_EC(expanded_labeled_trips,False, unit_dist_MCS_df, android_EI_moments_df, ios_EI_moments_df, gis_sensed_modes,energy_dict,False,False)
prediction_EC_primary_trip[0],user_EC_trip[0],confusion_EC_primary_trip[0]

In [ ]:
np.diff([prediction_EC_primary_trip[0],user_EC_trip[0],confusion_EC_primary_trip[0]])

In [ ]:
# drop air trips
# Constructing the propagation function.

use_sensing_only = True

# probably should get segments ahead of time and store in the database.

# get_aggregate_EC(trips_df, only_sensing, unit_dist_MCS, android_EI_moments, ios_EI_moments,gis_sensed_modes, energy_dict):
mean_EC_all_sensing_vail, var_EC_all_sensing_vail = get_aggregate_EC(expanded_labeled_trips,use_sensing_only, unit_dist_MCS_df, android_EI_moments_df, ios_EI_moments_df, gis_sensed_modes,energy_dict)

mean_EC_all_user_labeled_vail, var_EC_all_user_labeled_vail = get_aggregate_EC(expanded_labeled_trips,False, unit_dist_MCS_df, android_EI_moments_df, ios_EI_moments_df, gis_sensed_modes,energy_dict)

In [ ]:
# with no shared rides.
elt = expanded_labeled_trips.copy()
no_shared_rides = elt.drop(elt[elt.mode_confirm == "shared_ride"].index)

# use a confusion matrix that has 
android_EI_moments_no_shared_ride, ios_EI_moments_no_shared_ride = (pd.read_csv("android_EI_moments_no_shared_ride.csv").set_index("mode"),pd.read_csv("ios_EI_moments_no_shared_ride.csv").set_index("mode"))
mean_EC_no_shared, var_EC_no_shared = get_aggregate_EC(no_shared_rides,True, unit_dist_MCS_df, android_EI_moments_no_shared_ride, ios_EI_moments_no_shared_ride, gis_sensed_modes,energy_dict)
mean_EC_no_shared_users, var_EC_no_shared_users = get_aggregate_EC(no_shared_rides,False, unit_dist_MCS_df, android_EI_moments_no_shared_ride, ios_EI_moments_no_shared_ride, gis_sensed_modes,energy_dict)
mean_EC_naive_no_shared, var_EC_naive_no_shared, avg_EI_no_shared= get_aggregate_EC_with_extras(no_shared_rides,True, unit_dist_MCS_df, android_EI_moments_no_shared_ride, ios_EI_moments_no_shared_ride, gis_sensed_modes,energy_dict,use_naive_sensing_prediction=True)
print(f"EC from: sensing with confusion, sensing prediction, user labels:{mean_EC_no_shared,mean_EC_naive_no_shared,mean_EC_no_shared_users}")

In [ ]:
mean_EC_naive_no_shared

In [ ]:
mean_EC_naive_no_shared, var_EC_naive_no_shared, avg_EI_no_shared= get_aggregate_EC_with_extras(no_shared_rides,True, unit_dist_MCS_df, android_EI_moments_no_shared_ride, ios_EI_moments_no_shared_ride, gis_sensed_modes,energy_dict,use_naive_sensing_prediction=True,car_load_factor = 1)

In [ ]:
def get_aggregate_EC_with_extras(trips_df, only_sensing, unit_dist_MCS, android_EI_moments, ios_EI_moments,gis_sensed_modes, energy_dict, use_naive_sensing_prediction=False, car_load_factor=1.5):
    # requires the trips dataframe to have expanded labeled trips

    # The load factor here only updates the predicted mode EI, not the confusion EI. See store_errors.ipynb to save EI_moments dataframes with different load factors.
    drove_alone_EI = energy_dict["Gas Car, drove alone"]
    energy_dict.update({"Gas Car, sensed": drove_alone_EI/car_load_factor})

    mean_EC_agg = 0    # aggregate energy consumption
    var_EC_agg = 0  
    sum_sensed_mean_EI = 0
    N_sections = 0

    sum_labeled_mean_EI = 0
    n_trips = 0

    ios_count = 0
    android_count = 0

    for  _,ct in trips_df.iterrows():
        # Get operating system
        u = ecwu.User(ct.user_id)
        os = u.getProfile()["curr_platform"]

        if os == "ios": 
            ios_count+=1 
        else: 
            android_count += 1

        # Get OS specific trip length info.
        mean_for_unit_L = unit_dist_MCS_df[os]["mean"]
        var_for_unit_L = unit_dist_MCS_df[os]["var"]


        # Get trip mode info.
        # Later the condition will be whether the model chosen is sensing.
        if only_sensing == True:
            # Get segments for the trip.
            n_sections = len(ct["section_modes"])
            section_modes = ct["section_modes"]
            sections_lengths = np.array(ct["section_distances"])*METERS_TO_MILES   # 1 meter = 0.000621371 miles

            mean_L = sections_lengths*mean_for_unit_L
            var_L = sections_lengths**2 * var_for_unit_L
            
            for s in range(0,n_sections):
                # EI mean and variance.
                # Perhaps it would be better to keep the moments in the same file?

                if section_modes[s] == "air_or_hsr": continue

                if use_naive_sensing_prediction:
                    #mean_EI = energy_dict[MODE_MAPPING_DICT[section_modes[s]]]
                    if section_modes[s] == 'car':
                        mean_EI = energy_dict['Gas Car, sensed']
                    else:
                        mean_EI = energy_dict[MODE_MAPPING_DICT[section_modes[s]]]
                    var_EI = 0
                else:
                    # Later: switch to a map style function.
                    mean_EI, var_EI = get_EI_moments_for_trip(section_modes[s],os,android_EI_moments,ios_EI_moments)

                sum_sensed_mean_EI += mean_EI
                N_sections += 1

                # Propagate variance for the trip
                mean_EC = mean_L[s]*mean_EI
                var_EC = var_EI*mean_L[s]**2 + var_L[s]*mean_EI**2

                # Add to total - follows from assumed independence of section errors.
                mean_EC_agg += mean_EC
                var_EC_agg += var_EC
        
        # use user labels.
        else:
            mode = ct["mode_confirm"]  # need to make sure you convert it to an appropriate energy intensity.

            if mode not in MODE_MAPPING_DICT or mode == np.nan: continue
            if MODE_MAPPING_DICT[mode] == "Air": continue
            EI = energy_dict[MODE_MAPPING_DICT[mode]]

            sum_labeled_mean_EI += EI
            n_trips += 1

            length = ct["distance"]*METERS_TO_MILES
            mean_L = length* mean_for_unit_L  
            var_L = length**2 * var_for_unit_L

            mean_EC_agg += EI*mean_L
            var_EC_agg +=  EI*var_L

        avg_EI = sum_sensed_mean_EI/N_sections if only_sensing == True else sum_labeled_mean_EI/n_trips

    #print(f"ios vs android trip count: {ios_count,android_count}")
    #print(f"Sum of EIs (sensed, user labeled): {sum_sensed_mean_EI,sum_labeled_mean_EI}")   # could weight by distance
    #print(f"number of sections or trips: {N_sections,n_trips}")
    #print(air_count)
    return mean_EC_agg, var_EC_agg, avg_EI

#mean_EC_naive_sensing_vail, var_EC_naive_sensing_vail, avg_EI_sensed_naive = get_aggregate_EC_with_extras(expanded_labeled_trips,use_sensing_only, unit_dist_MCS_df, android_EI_moments_df, ios_EI_moments_df, gis_sensed_modes,energy_dict,use_naive_sensing_prediction=True)

In [ ]:
# Split the data.
# 10^3 NMC takes 10 seconds on vail to create all 4 splits.
proportion_sensed = [0.2,0.4,0.6,0.8]
NMC = 10#**2#**3

summary_df_map = {}
for ps in proportion_sensed:
    
    mean_EC_agg = []
    var_EC_agg = []
    for j in range(0,NMC):
        rand_state = np.random.RandomState(1+j)

        # Split the labeled trips into a user labeled dataframe and a sensed dataframe
        user_labeled,sensed  = skm.train_test_split(expanded_labeled_trips, 
                                                    test_size = ps, # sensed
                                                    train_size = 1-ps,  # user_labeled
                                                    random_state= rand_state)
        mean_EC_sensed, var_EC_sensed = \
            get_aggregate_EC(sensed,True, unit_dist_MCS_df, android_EI_moments_df, ios_EI_moments_df, gis_sensed_modes,energy_dict)
        
        mean_EC_user_labeled, var_EC_user_labeled = \
            get_aggregate_EC(user_labeled,False, unit_dist_MCS_df, android_EI_moments_df, ios_EI_moments_df, gis_sensed_modes,energy_dict)

        mean_EC_agg.append(mean_EC_sensed + mean_EC_user_labeled)
        var_EC_agg.append(var_EC_sensed + var_EC_user_labeled)

        sd_EC_agg = np.sqrt(np.array(var_EC_agg))

        summary_df_map[ps] = pd.DataFrame({"mean": mean_EC_agg, "sd": sd_EC_agg})
        
        # prop var sensed
        # prop var user labeled

In [ ]:
summary_df_map[0.2] # could look at proportions of each sensed mode?

In [ ]:
average_summaries = {}
for ps in proportion_sensed:
    average_across_splits_mean = np.mean(summary_df_map[ps]["mean"])
    average_across_splits_sd = np.mean(summary_df_map[ps]["sd"])
    average_summaries[ps] = {"mean": average_across_splits_mean, "sd": average_across_splits_sd}
average_summaries

In [ ]:
mean_EC_all_sensing_vail

In [ ]:
sd_sensed = np.sqrt(var_EC_all_sensing_vail)
sd_users = np.sqrt(var_EC_all_user_labeled_vail)

def get_interval(mean,sd):
    return [mean -sd, mean,mean + sd]

interval_sensed_vail = get_interval(mean_EC_all_sensing_vail,sd_sensed)
interval_users_vail = get_interval(mean_EC_all_user_labeled_vail,sd_users)

In [ ]:
interval_sensed_vail

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots()
fig.set_figheight(6)

ax.plot([0]*3,interval_users_vail,'bo') 
j = 1
for ps in proportion_sensed:
    summary = average_summaries[ps]
    x = [ps]*3
    y = get_interval(summary["mean"],summary["sd"])
    ax.plot(x,y,'bo')
    j+=1
ax.plot([1]*3,interval_sensed_vail,'bo')
ax.set_ylim([4000,12000])
ax.set_xlabel("Proportion of trips using sensing as opposed to user labels")

fig.suptitle("Energy consumption mean +- 1 sd as percent of sensed trips increases")

In [ ]:
# horizontal version of the plots
import matplotlib.pyplot as plt
fig,axs = plt.subplots(6,sharex=True)
fig.set_figheight(10)
y = [0,0,0]
axs[0].plot(interval_users_vail,y,'bo') 
j = 1
for ps in proportion_sensed:
    summary = average_summaries[ps]
    x = [ps]*3
    y = get_interval(summary["mean"],summary["sd"])
    axs[j].plot(x,y,'bo')
    j+=1
axs[5].plot(interval_sensed_vail,y,'bo')

fig.suptitle("0 -> 1 proportion sensed in increments of 0.2 (mean +- 1 sd)")



In [ ]:
sd_EC_agg = np.sqrt(np.array(var_EC_agg))

summary_df = pd.DataFrame({"mean": mean_EC_agg, "sd": sd_EC_agg}).set_index(proportion_sensed)